In [1]:
import os
import functools
from encode import encode_helpers
# from gscripts.encode import encode_helpers # this is from gscripts, but having trouble with import after install :/

actual_peaks_dir = "/projects/ps-yeolab3/encode/analysis/Eric_IDR/"

# Fetch all eCLIP file locations

In [2]:
merged_data = encode_helpers.get_merged_data()

#For Peak Analysis we only want to analyze datasets that have been submitted 
#(we can talk about other ways to filter if you want, but these are the most confident datasets)
merged_data = merged_data[merged_data.submitted]


#merged_data = merged_data[['CLIP', 'INPUT', 'input_norm']]
make_and_filter_clipper_moderate = functools.partial(encode_helpers.make_and_filter_clipper, l2fc=3, pval=3)
merged_data['filtered_peaks'] = merged_data['input_norm'].apply(make_and_filter_clipper_moderate)

## Column Information for data frame

- uID - internal unique ID for expermental triple (rep1, rep2, size matched input)
- Cell line - cell line experiment was done in
- RBP - RNA Binding Protein clipped
– CLIP - bam file for the clip rep
- INPUt - bam file for the input.  For both reps this will be the same
- input norm - input normalized (but not filtered) bed file.  Contains all peaks no matter signifance value
- filtered_peaks - peaks that pass signifiance threshold (log2 fold change > 3 and -log10 p-value > 3)

In [3]:
merged_data.head()

CLIP                                                                     CLIP  \
uID Cell line RBP     rep                                                       
203 HepG2     HNRNPC  rep1  /projects/ps-yeolab3/encode/analysis/encode_ma...   
                      rep2  /projects/ps-yeolab3/encode/analysis/encode_ma...   
204 HepG2     RBFOX2  rep1  /projects/ps-yeolab3/encode/analysis/encode_ma...   
                      rep2  /projects/ps-yeolab3/encode/analysis/encode_ma...   
205 HepG2     IGF2BP1 rep1  /projects/ps-yeolab3/encode/analysis/encode_ma...   

CLIP                          \
uID Cell line RBP     rep      
203 HepG2     HNRNPC  rep1     
                      rep2     
204 HepG2     RBFOX2  rep1     
                      rep2     
205 HepG2     IGF2BP1 rep1     

CLIP                                                                    INPUT  \
uID Cell line RBP     rep                                                       
203 HepG2     HNRNPC  rep1  /projects/ps-yeolab3/encode/analysis/encode_ma...   
                      rep2  /projects/ps-yeolab3/encode/analysis/encode_ma...   
204 HepG2     RBFOX2  rep1  /projects/ps-yeolab3/encode/analysis/encode_ma...   
                      rep2  /projects/ps-yeolab3/encode/analysis/encode_ma...   
205 HepG2     IGF2BP1 rep1  /projects/ps-yeolab3/encode/analysis/encode_ma...   

CLIP                       Submitted Date annotation family family_bound  \
uID Cell line RBP     rep                                                  
203 HepG2     HNRNPC  rep1     2015-09-16                          FALSE   
                      rep2     2015-09-16                          FALSE   
204 HepG2     RBFOX2  rep1     2015-02-18                          FALSE   
                      rep2     2015-02-18                          FALSE   
205 HepG2     IGF2BP1 rep1     2015-02-18                          FALSE   

CLIP                       family_mapping_submitable is_qced note  \
uID Cell line RBP     rep                                           
203 HepG2     HNRNPC  rep1                     False    True        
                      rep2                     False    True        
204 HepG2     RBFOX2  rep1                     False    True        
                      rep2                     False    True        
205 HepG2     IGF2BP1 rep1                     False    True        

CLIP                                                  notes peaks_submitable  \
uID Cell line RBP     rep                                                      
203 HepG2     HNRNPC  rep1  Intronic, UGU motif? binds alus             True   
                      rep2  Intronic, UGU motif? binds alus             True   
204 HepG2     RBFOX2  rep1           Intronic, UGCAUG motif             True   
                      rep2           Intronic, UGCAUG motif             True   
205 HepG2     IGF2BP1 rep1                         CDS/3UTR             True   

CLIP                       reference submitted generally_submittable  \
uID Cell line RBP     rep                                              
203 HepG2     HNRNPC  rep1                True                  True   
                      rep2                True                  True   
204 HepG2     RBFOX2  rep1                True                  True   
                      rep2                True                  True   
205 HepG2     IGF2BP1 rep1                True                  True   

CLIP                                                               input_norm  \
uID Cell line RBP     rep                                                       
203 HepG2     HNRNPC  rep1  /projects/ps-yeolab3/encode/analysis/Eric_Inpu...   
                      rep2  /projects/ps-yeolab3/encode/analysis/Eric_Inpu...   
204 HepG2     RBFOX2  rep1  /projects/ps-yeolab3/encode/analysis/Eric_Inpu...   
                      rep2  /projects/ps-yeolab3/encode/analysis/Eric_Inpu...   
205 HepG2     IGF2BP1 rep1  /projects/ps-yeolab3/encode/analysis/Eric_

# Format of eCLIP bed file

- Col 1: chrom
- Col 2: start
- Col 3: stop
- Col 4: log2 fold change
- Col 5: -log10 p-value
- Col 6: strand
- Col 7: annotation
- Col 8: gene ID

In [4]:
foo = merged_data['input_norm'][0]

!head -n 2 $foo

chr7	138151219	138151300	23.6215112519682	3.45382390435692	+	intron|contained|ENSG00000122779.12	ENSG00000122779.12
chr7	139051735	139051798	15.3672998138034	3.7517921003001	+	intron|contained|ENSG00000146963.13:ENSG00000269955.2	ENSG00000146963.13|ENSG00000269955.2


# Construct IDR manifest


In [5]:
rep_information = ['CLIP', 'input_norm']
temp_index = list(merged_data.columns.difference(rep_information))
idr_manifest = merged_data.xs("rep1", level="rep")
idr_manifest = idr_manifest.set_index(temp_index, append=True)
idr_manifest = idr_manifest.reset_index()
idr_manifest = idr_manifest.set_index(merged_data.index.names[:-1])
idr_manifest['uID'] = idr_manifest.index.get_level_values("uID")

idr_manifest['replicate_merged'] = idr_manifest.uID.apply(lambda x: os.path.join(actual_peaks_dir, x + ".01v02.IDR.out.0102merged.bed"))

# Column information for dataframe 
good luck its similar to the other one, just with one col per experiment. 

replicate_merged_v2 is the important column its got the IDR peaks

In [6]:
idr_manifest.head()

,,CLIP,,INPUT,Submitted Date,annotation,family,family_bound,family_mapping_submitable,filtered_peaks,generally_submittable,is_qced,note,notes,peaks_submitable,reference,submitted,CLIP,input_norm,uID,replicate_merged
uID,Cell line,RBP,,,,,,,,,,,,,,,,,,,
203,HepG2,HNRNPC,,/projects/ps-yeolab3/encode/analysis/encode_ma...,2015-09-16,,,FALSE,False,/home/gpratt/projects/encode/analysis/peak_rea...,True,True,,"Intronic, UGU motif? binds alus",True,,True,/projects/ps-yeolab3/encode/analysis/encode_ma...,/projects/ps-yeolab3/encode/analysis/Eric_Inpu...,203,/projects/ps-yeolab3/encode/analysis/Eric_IDR/...
204,HepG2,RBFOX2,,/projects/ps-yeolab3/encode/analysis/encode_ma...,2015-02-18,,,FALSE,False,/home/gpratt/projects/encode/analysis/peak_rea...,True,True,,"Intronic, UGCAUG motif",True,,True,/projects/ps-yeolab3/encode/analysis/encode_ma...,/projects/ps-yeolab3/encode/analysis/Eric_Inpu...,204,/projects/ps-yeolab3/encode/analysis/Eric_IDR/...
205,HepG2,IGF2BP1,,/projects/ps-yeolab3/encode/analysis/encode_ma...,2015-02-18,,,FALSE,False,/home/gpratt/projects/encode/analysis/peak_rea...,True,True,,CDS/3UTR,True,,True,/projects/ps-yeolab3/encode/analysis/encode_ma...,/projects/ps-yeolab3/encode/analysis/Eric_Inpu...,205,/projects/ps-yeolab3/encode/analysis/Eric_IDR/...
206,HepG2,HNRNPK,,/projects/ps-yeolab3/encode/analysis/encode_ma...,2015-02-23,,,FALSE,False,/home/gpratt/projects/encode/analysis/peak_rea...,True,True,,"Intronic (dist+prox), CCC motif",True,,True,/projects/ps-yeolab3/encode/analysis/encode_ma...,/projects/ps-yeolab3/encode/analysis/Eric_Inpu...,206,/projects/ps-yeolab3/encode/analysis/Eric_IDR/...
209,HepG2,SRSF7,,/projects/ps-yeolab3/encode/analysis/encode_ma...,2015-02-23,,,FALSE,False,/home/gpratt/projects/encode/analysis/peak_rea...,True,True,,top binding site is 3' utr extension?,True,,True,/projects/ps-yeolab3/encode/analysis/encode_ma...,/projects/ps-yeolab3/encode/analysis/Eric_Inpu...,209,/projects/ps-yeolab3/encode/analysis/Eric_IDR/...


# Format of IDR bed file

- Col 1: chrom
- Col 2: start
- Col 3: stop
- Col 4: log2 fold change
- Col 5: -log10 p-value
- Col 6: strand

In [7]:
foo = idr_manifest['replicate_merged'][0]

!head -n 2 $foo

chr1	247086670	247086708	5.15157944416774	3.53270919042856	-
chr1	247086708	247086720	3.40206294870218	3.09777048796747	-


# Save the IDR manifest

In [8]:
del idr_manifest['uID']
idr_manifest.reset_index(inplace=True)
idr_manifest.to_csv(
    '/home/bay001/projects/maps_20160420/permanent_data/ALLDATASETS.txt', 
    sep='\t', header=True, index=False, encoding='utf-8'
)